# Welcome to dfsql!

[dfsql](https://github.com/mindsdb/dfsql) allows you to query Pandas and Modin DataFrames using SQL queries.

This notebook will guide you through the main features.

In [5]:
import os
import pandas
import pandas as pd

In [6]:
import warnings
# Filter Modin userwarnings about defaulting to pandas implementations
warnings.filterwarnings('ignore', category=UserWarning)

In [1]:
# Comment out if you don't have modin installed
%env USE_MODIN=True
import modin.pandas as pd
from dfsql.config import Configuration
Configuration.as_dict()

env: USE_MODIN=True


{'USE_MODIN': 1}

In [2]:
from dfsql import DataSource

In [3]:
%load_ext autoreload
%autoreload 2

# Loading data

In [7]:
os.makedirs('testdrive_csvs', exist_ok=True)

In [8]:
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore.csv -o testdrive_csvs/googleplaystore.csv
!curl https://raw.githubusercontent.com/jasonchang0/kaggle-google-apps/master/google-play-store-apps/googleplaystore_user_reviews.csv -o testdrive_csvs/googleplaystore_user_reviews.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1328k  100 1328k    0     0  1953k      0 --:--:-- --:--:-- --:--:-- 1953k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7489k  100 7489k    0     0  3382k      0  0:00:02  0:00:02 --:--:-- 3382k


## The dataset contains google play apps and user reviews

In [9]:
googleplaystore = pd.read_csv('testdrive_csvs/googleplaystore.csv')
googleplaystore.head()

App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend Play  January 15, 2018               2.0.0   
2               Art & Design    August 1, 2018               1.2.4   
3               Art & Design      June 8, 2018  Varies with device   
4    Art & Design;Creativity     June 20, 2018                 1.1   

    Android Ver  
0  4.0.3 and up  
1  4.0.3 and up  
2  4.0.3 and up  
3    4.2 and up  
4    4.4 and up

In [10]:
googleplaystore_user_reviews = pd.read_csv('testdrive_csvs/googleplaystore_user_reviews.csv')
googleplaystore_user_reviews.head()

App                                  Translated_Review  \
0  10 Best Foods for You  I like eat delicious food. That's I'm cooking ...   
1  10 Best Foods for You    This help eating healthy exercise regular basis   
2  10 Best Foods for You                                                NaN   
3  10 Best Foods for You         Works great especially going grocery store   
4  10 Best Foods for You                                       Best idea us   

  Sentiment  Sentiment_Polarity  Sentiment_Subjectivity  
0  Positive                1.00                0.533333  
1  Positive                0.25                0.288462  
2       NaN                 NaN                     NaN  
3  Positive                0.40                0.875000  
4  Positive                1.00                0.300000

In [11]:
googleplaystore.Price

0        0
1        0
2        0
3        0
4        0
        ..
10836    0
10837    0
10838    0
10839    0
10840    0
Name: Price, Length: 10841, dtype: object

# Quick interfaces

## sql_query

`sql_query` takes as input the sql string and Pandas or Modin dataframes as keywords.

In [12]:
from dfsql import sql_query

In [13]:
result_df = sql_query("SELECT App, Category, Rating, Price FROM table1 WHERE Price = \"0\" LIMIT 5", 
                      table1=googleplaystore)
result_df

App        Category Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN    4.1   
1                                Coloring book moana  ART_AND_DESIGN    3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN    4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN    4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN    4.3   

  Price  
0     0  
1     0  
2     0  
3     0  
4     0

Multiple tables can be passed to perform joins and complex queries.

In [18]:
sql = """
SELECT App, avg(Sentiment_Polarity) as avg_sentiment_polarity
FROM (
    SELECT App, Sentiment, CAST(Sentiment_Polarity as float) as Sentiment_Polarity
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews 
    ON googleplaystore.App = googleplaystore_user_reviews.App 
) AS sub
GROUP BY App
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
result_df = sql_query(sql, googleplaystore=googleplaystore, googleplaystore_user_reviews=googleplaystore_user_reviews)
result_df

App  avg_sentiment_polarity
0                              10 Best Foods for You                0.470733
4   2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif                0.449566
8                                          2RedBeans                0.412199
11                           365Scores - Live Scores                0.438312
14                        3D Live Neon Weed Launcher                0.568182
19                                  7 Minute Workout                0.410278
22                            850 Sports News Digest                0.542860
43                      APE Weather ( Live Forecast)                0.432323
44       APUS Launcher - Theme, Wallpaper, Hide Apps                0.447473
52                               ASUS Sound Recorder                0.516771

## Pandas/Modin extensions


`dataskillet.extensions` adds the `.sql` method to Pandas and Modin dataframes when imported. 

This method takes only an sql query. It's useful when only one table is used in the query.

In [19]:
# Make sure to import it **AFTER** Pandas or Modin
import dfsql.extensions

### Example with a Modin DataFrame

In [29]:
print(type(googleplaystore))
googleplaystore.sql("SELECT App, Category") # Alternatively: "SELECT App, Category FROM temp"

<class 'pandas.core.frame.DataFrame'>


App             Category
0         Photo Editor & Candy Camera & Grid & ScrapBook       ART_AND_DESIGN
1                                    Coloring book moana       ART_AND_DESIGN
2      U Launcher Lite – FREE Live Cool Themes, Hide ...       ART_AND_DESIGN
3                                  Sketch - Draw & Paint       ART_AND_DESIGN
4                  Pixel Draw - Number Art Coloring Book       ART_AND_DESIGN
...                                                  ...                  ...
10836                                   Sya9a Maroc - FR               FAMILY
10837                   Fr. Mike Schmitz Audio Teachings               FAMILY
10838                             Parkinson Exercices FR              MEDICAL
10839                      The SCP Foundation DB fr nn5n  BOOKS_AND_REFERENCE
10840      iHoroscope - 2018 Daily Horoscope & Astrology            LIFESTYLE

[10841 rows x 2 columns]

### Example with a Pandas DataFrame

The operations are still executed in Modin if it's enabled.

In [30]:
googleplaystore_pandas = pandas.read_csv('testdrive_csvs/googleplaystore.csv')

print(type(googleplaystore_pandas))
googleplaystore_pandas.sql("SELECT App, Category")

<class 'pandas.core.frame.DataFrame'>


App             Category
0         Photo Editor & Candy Camera & Grid & ScrapBook       ART_AND_DESIGN
1                                    Coloring book moana       ART_AND_DESIGN
2      U Launcher Lite – FREE Live Cool Themes, Hide ...       ART_AND_DESIGN
3                                  Sketch - Draw & Paint       ART_AND_DESIGN
4                  Pixel Draw - Number Art Coloring Book       ART_AND_DESIGN
...                                                  ...                  ...
10836                                   Sya9a Maroc - FR               FAMILY
10837                   Fr. Mike Schmitz Audio Teachings               FAMILY
10838                             Parkinson Exercices FR              MEDICAL
10839                      The SCP Foundation DB fr nn5n  BOOKS_AND_REFERENCE
10840      iHoroscope - 2018 Daily Horoscope & Astrology            LIFESTYLE

[10841 rows x 2 columns]

# In-depth usage

Under the hood `sql_query` creates a DataSource, adds tables to it, executes the query, and then tears everything down. This is not very efficient if many queries have to be executed against the same tables. In that case it's better to use DataSource directly.

DataSource is the analogus of database. It stores information about tables, loads tables from disk when they are needed, handles caching. 
It accepts queries like a database would.

## Creating the DataSource
On creation, we need to specify a directory to store table metadata for the DataSource.

In [31]:
metadata_dir = 'testdrive_metadata'
os.mkdir(metadata_dir)

In [32]:
ds = DataSource(metadata_dir=metadata_dir)

The metadata dir now stores information about tables. However we currently have no tables. Lets create some

In [33]:
ds.query('SHOW TABLES')

Empty DataFrame
Columns: [name, fpath]
Index: []

## Creating tables

A `Table` is an abstraction over a dataframe. It loads the dataframe on-demand. On creation it applies simple preprocessing by converting dtypes.

In [34]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore.csv")')

'OK'

In [35]:
ds.query(f'CREATE TABLE ("testdrive_csvs/googleplaystore_user_reviews.csv")')

'OK'

In [36]:
ds.query('SHOW TABLES')

name  \
0               googleplaystore   
1  googleplaystore_user_reviews   

                                             fpath  
0               testdrive_csvs/googleplaystore.csv  
1  testdrive_csvs/googleplaystore_user_reviews.csv

Now that we have some tables, the information about them is stored in metadata. 

If we recreate the datasource using the same `metadata_dir`, **we don't need to add the tables again**.

In [37]:
ds = DataSource(metadata_dir=metadata_dir)

In [38]:
ds.query('SHOW TABLES')

name  \
0               googleplaystore   
1  googleplaystore_user_reviews   

                                             fpath  
0               testdrive_csvs/googleplaystore.csv  
1  testdrive_csvs/googleplaystore_user_reviews.csv

If we need it, we can clear the metadata and recreate the DataSource using `DataSource.create_new(metadata_dir)`

## Querying

In [39]:
ds.query('SELECT * FROM googleplaystore LIMIT 5')

App        Category Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN    4.1   
1                                Coloring book moana  ART_AND_DESIGN    3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN    4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN    4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN    4.3   

  Reviews  Size     Installs  Type Price `Content Rating`  \
0     159   19M      10,000+  Free     0         Everyone   
1     967   14M     500,000+  Free     0         Everyone   
2   87510  8.7M   5,000,000+  Free     0         Everyone   
3  215644   25M  50,000,000+  Free     0             Teen   
4     967  2.8M     100,000+  Free     0         Everyone   

                      Genres    `Last Updated`       `Current Ver`  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend Play  January 15, 2018               2.0.0   
2               Art & Design    August 1, 2018               1.2.4   
3               Art & Design      June 8, 2018  Varies with device   
4    Art & Design;Creativity     June 20, 2018                 1.1   

  `Android Ver`  
0  4.0.3 and up  
1  4.0.3 and up  
2  4.0.3 and up  
3    4.2 and up  
4    4.4 and up

In [41]:
ds.query("SELECT App FROM googleplaystore LIMIT 5")

0       Photo Editor & Candy Camera & Grid & ScrapBook
1                                  Coloring book moana
2    U Launcher Lite – FREE Live Cool Themes, Hide ...
3                                Sketch - Draw & Paint
4                Pixel Draw - Number Art Coloring Book
Name: App, dtype: object

In [42]:
ds.query("SELECT CAST(Price as int) as price_int FROM googleplaystore WHERE Price = \"0\" LIMIT 5")

0    0
1    0
2    0
3    0
4    0
Name: price_int, dtype: int64

In [ ]:
ds.query("SELECT App, Category, `Content Rating` FROM googleplaystore WHERE Price = \"0\"")

In [47]:
ds.query("SELECT App, Category, Reviews FROM googleplaystore WHERE Category = \"FAMILY\" AND Price = \"0\" ORDER BY App ASC LIMIT 100")

App Category Reviews
1228                10 Minutes a Day Times Tables   FAMILY     681
1430                         100 Doors of Revenge   FAMILY  105766
259                               100+ C Programs   FAMILY      20
255                    101 C Programming Problems   FAMILY     498
306              2048 BTS J Hope KPop Puzzle Game   FAMILY      17
...                                           ...      ...     ...
1234  Account Class-12 Solutions (D K Goel) Vol-2   FAMILY     124
572                 Accounting Quiz (AQ) Malaysia   FAMILY      25
1319                                Across Age DX   FAMILY    5964
444                       Ad-free Triangle Solver   FAMILY     118
1644           Adhenarcos - Coupe Adhemar EY 2018   FAMILY       0

[100 rows x 3 columns]

In [48]:
ds.query("SELECT App, Sentiment, Sentiment_Polarity FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.App = googleplaystore_user_reviews.App LIMIT 10")

App Sentiment Sentiment_Polarity
0  Coloring book moana  Negative              -0.25
1  Coloring book moana  Negative             -0.725
2  Coloring book moana   Neutral                0.0
3  Coloring book moana      <NA>               <NA>
4  Coloring book moana  Positive                0.5
5  Coloring book moana  Negative               -0.8
6  Coloring book moana      <NA>               <NA>
7  Coloring book moana   Neutral                0.0
8  Coloring book moana  Positive                0.5
9  Coloring book moana  Positive                0.5

In [50]:
sql = """
SELECT App, avg(Sentiment_Polarity) as avg_sentiment_polarity
FROM (
    SELECT App, Sentiment, CAST(Sentiment_Polarity as float) as Sentiment_Polarity
    FROM googleplaystore INNER JOIN googleplaystore_user_reviews ON googleplaystore.App = googleplaystore_user_reviews.App 
) AS sub
GROUP BY App
HAVING CAST(avg_sentiment_polarity as float) > 0.4
LIMIT 10
"""
ds.query(sql)

App  avg_sentiment_polarity
0                              10 Best Foods for You                0.470733
4   2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif                0.449566
8                                          2RedBeans                0.412199
11                           365Scores - Live Scores                0.438312
14                        3D Live Neon Weed Launcher                0.568182
19                                  7 Minute Workout                0.410278
22                            850 Sports News Digest                0.542860
43                      APE Weather ( Live Forecast)                0.432323
44       APUS Launcher - Theme, Wallpaper, Hide Apps                0.447473
52                               ASUS Sound Recorder                0.516771

In [51]:
sql = """
SELECT count(App) as count_apps
FROM (
    SELECT App
    FROM googleplaystore 
) AS sub
"""
ds.query(sql)

10841

In [53]:
sql = """
SELECT Category, count(App) as count_apps
FROM (
    SELECT Category, App FROM googleplaystore 
) AS sub
GROUP BY Category
ORDER BY count_apps DESC
LIMIT 10
"""
ds.query(sql)

Category  count_apps
12           FAMILY        1972
15             GAME        1144
30            TOOLS         843
21          MEDICAL         463
5          BUSINESS         460
26     PRODUCTIVITY         424
24  PERSONALIZATION         392
7     COMMUNICATION         387
29           SPORTS         384
19        LIFESTYLE         382

In [54]:
sql = """
SELECT count(Category) as uniq_categories
FROM (
    SELECT DISTINCT Category FROM googleplaystore 
) AS sub
"""
ds.query(sql)

34

In [55]:
sql = """
SELECT count(distinct Category) as uniq_categories
FROM googleplaystore 
"""
ds.query(sql)

34

In [56]:
tdf = ds.query("SELECT App || '_app_name'   FROM googleplaystore LIMIT 5")
tdf[0]

'Photo Editor & Candy Camera & Grid & ScrapBook_app_name'

In [57]:
ds.query("SELECT upper(App), Category FROM googleplaystore WHERE App LIKE \".*best.*\" LIMIT 5")

upper(App) Category
0  KLART.SE - SWEDEN'S BEST WEATHER  WEATHER

In [61]:
ds.query("SELECT App, Sentiment, Translated_Review FROM googleplaystore_user_reviews WHERE Translated_Review IS NULL")

App Sentiment Translated_Review
0            10 Best Foods for You      <NA>              <NA>
1            10 Best Foods for You      <NA>              <NA>
2            10 Best Foods for You      <NA>              <NA>
3            10 Best Foods for You      <NA>              <NA>
4            10 Best Foods for You      <NA>              <NA>
...                            ...       ...               ...
26863  Houzz Interior Design Ideas      <NA>              <NA>
26864  Houzz Interior Design Ideas      <NA>              <NA>
26865  Houzz Interior Design Ideas      <NA>              <NA>
26866  Houzz Interior Design Ideas      <NA>              <NA>
26867  Houzz Interior Design Ideas      <NA>              <NA>

[26868 rows x 3 columns]

In [62]:
ds.query("SELECT App, Sentiment FROM googleplaystore_user_reviews WHERE Sentiment IN ('Positive', 'Negative')")

App Sentiment
0               10 Best Foods for You  Positive
1               10 Best Foods for You  Positive
2               10 Best Foods for You  Positive
3               10 Best Foods for You  Positive
4               10 Best Foods for You  Positive
...                               ...       ...
32264  Housing-Real Estate & Property  Positive
32265  Housing-Real Estate & Property  Positive
32266  Housing-Real Estate & Property  Negative
32267  Housing-Real Estate & Property  Positive
32268  Housing-Real Estate & Property  Negative

[32269 rows x 2 columns]

## Dataframe caching

Dataskillet caches dataframes in memory to speed up queries against the same tables.

First let's clear the cache

In [63]:
ds.cache.clear()

Querying with the cache clear

In [64]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

CPU times: user 477 ms, sys: 24 ms, total: 501 ms
Wall time: 769 ms


Check the cache

In [65]:
ds.cache.get.cache_info()

CacheInfo(hits=0, misses=1, maxsize=None, currsize=1)

Querying with the cache filled

In [66]:
%%time
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

CPU times: user 368 ms, sys: 11.7 ms, total: 380 ms
Wall time: 628 ms


Ensuring it was retrieved from cache:

In [67]:
ds.cache.get.cache_info()

CacheInfo(hits=1, misses=1, maxsize=None, currsize=1)

In [68]:
%%timeit
_ = ds.query('SELECT * FROM googleplaystore LIMIT 5')

642 ms ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
